<a href="https://colab.research.google.com/github/Hazem-A0/Shoplifting/blob/main/3DCNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Shop_DataSet.zip -d /content/drive/MyDrive/Shop_DataSet

Archive:  /content/drive/MyDrive/Shop_DataSet.zip
   creating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_0.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_1.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_10.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_10_1.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_100.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet/Shop DataSet/non shop lifters/shop_lifter_n_100_1.mp4  
  inflating: /content/drive/MyDrive/Shop_DataSet

In [20]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence
import multiprocessing
from functools import partial

def extract_frames(video_path, target_frames=10, target_size=(112, 112)):
    """Extract a fixed number of frames from a video, with reduced resolution."""
    frames = []
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        return None

    step = max(total_frames // target_frames, 1)
    frame_indices = range(0, min(total_frames, target_frames * step), step)

    for i in frame_indices:
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = video.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, target_size)
            frames.append(frame)

    video.release()

    # Pad or truncate to ensure we have exactly target_frames
    if len(frames) < target_frames:
        last_frame = frames[-1]
        frames.extend([last_frame] * (target_frames - len(frames)))
    elif len(frames) > target_frames:
        frames = frames[:target_frames]

    return np.array(frames)

def process_video(video_path, label, target_frames=10, target_size=(112, 112)):
    """Process a single video and return frames and label."""
    frames = extract_frames(video_path, target_frames, target_size)
    return frames, label

def load_video_paths(data_dir):
    """Load video paths and labels from the directory structure."""
    video_paths = []
    labels = []
    for label in ['shop_lifters', 'non_shop_lifters']:
        label_dir = os.path.join(data_dir, label)
        if not os.path.isdir(label_dir):
            print(f"Warning: {label_dir} not found.")
            continue
        for video_file in os.listdir(label_dir):
            if video_file.endswith(('.mp4', '.avi', '.mov')):
                video_paths.append(os.path.join(label_dir, video_file))
                labels.append(label)
    return video_paths, labels

class VideoDataGenerator(Sequence):
    def __init__(self, video_paths, labels, batch_size=32, target_frames=10, target_size=(112, 112)):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_frames = target_frames
        self.target_size = target_size
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_paths = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.encoded_labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        # Create a list of tuples (video_path, label) for the batch
        batch_data = list(zip(batch_paths, [self.labels[i] for i in batch_labels]))

        with multiprocessing.Pool() as pool:
            # Use pool.starmap to unpack the tuples and provide both arguments to process_video
            results = pool.starmap(partial(process_video,
                                          target_frames=self.target_frames,
                                          target_size=self.target_size),
                                  batch_data)

        batch_frames, _ = zip(*results)
        batch_frames = np.array(batch_frames).astype('float32') / 255.0
        batch_labels = to_categorical(batch_labels)

        return batch_frames, batch_labels

# Example usage:
data_dir = '/content/drive/MyDrive/Shop_DataSet/Shop_DataSet'
video_paths, labels = load_video_paths(data_dir)

# Create train and validation generators
from sklearn.model_selection import train_test_split

train_paths, val_paths, train_labels, val_labels = train_test_split(video_paths, labels, test_size=0.2, random_state=42)

train_generator = VideoDataGenerator(train_paths, train_labels)
val_generator = VideoDataGenerator(val_paths, val_labels)

print("Number of training samples:", len(train_paths))
print("Number of validation samples:", len(val_paths))

Number of training samples: 684
Number of validation samples: 171


In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, LSTM, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam

def create_3dcnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        # 3D CNN layers
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(1, 2, 2)),
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(1, 2, 2)),
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(1, 2, 2)),

        # Reshape for LSTM
        TimeDistributed(Flatten()),

        # LSTM layers
        LSTM(100, return_sequences=True),
        Dropout(0.2),
        LSTM(100),
        Dropout(0.2),

        # Dense layers
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Set up model parameters
input_shape = (10, 112, 112, 3)  # (time_steps, height, width, channels)
num_classes = 2

# Create the model
model = create_3dcnn_lstm_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_12 (Conv3D)                   │ (None, 8, 110, 110, 32)     │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_12 (MaxPooling3D)      │ (None, 8, 55, 55, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_13 (Conv3D)                   │ (None, 6, 53, 53, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_13 (MaxPooling3D)      │ (None, 6, 26, 26, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_14 (Conv3D)                   │ (None, 4, 24, 24, 64)       │         110,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_14 (MaxPooling3D)      │ (None, 4, 12, 12, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 4, 9216)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 4, 100)              │       3,726,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 4, 100)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,982,434 (15.19 MB)

 Trainable params: 3,982,434 (15.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy') # Change .h5 to .keras

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping],
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 485s 16s/step - accuracy: 0.5858 - loss: 0.6868 - val_accuracy: 0.6082 - val_loss: 0.6812
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 440s 15s/step - accuracy: 0.6299 - loss: 0.6738 - val_accuracy: 0.6082 - val_loss: 0.6724
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 439s 15s/step - accuracy: 0.6280 - loss: 0.6657 - val_accuracy: 0.6082 - val_loss: 0.6435
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 432s 14s/step - accuracy: 0.6644 - loss: 0.5652 - val_accuracy: 0.8596 - val_loss: 0.3322
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 498s 18s/step - accuracy: 0.8759 - loss: 0.3117 - val_accuracy: 0.9123 - val_loss: 0.2298
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 432s 14s/step - accuracy: 0.9082 - loss: 0.2385 - val_accuracy: 0.8304 - val_loss: 0.3328
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 432s 15s/step - accuracy: 0.8660 - loss: 0.3443 - val_accuracy: 0.9006 - val_loss: 0.2396
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 433s 15s/step - accuracy: 0.9288 - loss: 0.2109 - val_accuracy: 0.9298 - val_

Process ForkPoolWorker-635:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-20-0afc235bb2ba>", line 42, in process_video
    frames = extract_frames(video_path, target_frames, target_size)
  File "<ipython-input-20-0afc235bb2ba>", line 22, in extract_frames
    video.set(cv2.CAP_PROP_POS_FRAMES, i)
KeyboardInterrupt
Process ForkPoolWorker-636:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()